<a href="https://colab.research.google.com/github/harshelke180502/Big_Data_Mining-CS483-Harsh_Shelke/blob/main/Colab_1_Harsh_Shelke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark==3.3.2
# !pip install pyspark
# !pip install -U -q PyDrive
!pip uninstall -y PyDrive
!pip install -U PyDrive2
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824009 sha256=5239b26b9880b1219aa653619fe6050d2135ae21e7fb28f177608588efbf1d5d
  Stored in directory: /root/.cache/pip/wheels/93/99/74/4ba9a39bf51affed2cbf91a45688ea2a125775239338534f85
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.9
    Uninstalling py4j-0.10.9.9:
      Successfully uninstalled py4j-0.10.9.9
  Attempting uninstall: pyspark
    Found existing installation: pyspark 4.0.1
    Uninstalling pyspark-4.0.1:
      Successfully uninstalled pyspark-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [2]:
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java


update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [3]:
!java -version

openjdk version "1.8.0_472"
OpenJDK Runtime Environment (build 1.8.0_472-8u472-ga-1~22.04-b08)
OpenJDK 64-Bit Server VM (build 25.472-b08, mixed mode)


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [4]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id='1ChYlaWfcno8uzjsMhqFbJn5oc6roH1Ys'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [6]:
!java -version


openjdk version "1.8.0_472"
OpenJDK Runtime Environment (build 1.8.0_472-8u472-ga-1~22.04-b08)
OpenJDK 64-Bit Server VM (build 25.472-b08, mixed mode)


In [7]:
!ls /usr/lib/jvm


java-1.17.0-openjdk-amd64  java-1.8.0-openjdk-amd64
java-17-openjdk-amd64	   java-8-openjdk-amd64


In [8]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd


spark = SparkSession.builder.appName("Counting Word By StartingLetter").getOrCreate()



sc = spark.sparkContext

In [9]:

txt = spark.read.text("pg100.txt")
txt.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
| William Shakespeare|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|** This is a COPY...|
|**     Please fol...|
|                    |
|Title: The Comple...|
|                    |
|Author: William S...|
|                    |
|Posting Date: Sep...|
|Release Date: Jan...|
|                    |
|   Language: English|
|                    |
+--------------------+
only showing top 20 rows



In [10]:

words = txt.select(
    explode(
        split(col("value"), "\s+")
    ).alias("word")
)

words = words.select(lower(col("word")).alias("word"))

words.show()

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1225949128.py:4: SyntaxWarning: invalid escape sequence '\s'
  split(col("value"), "\s+")


+------------+
|        word|
+------------+
|         the|
|     project|
|   gutenberg|
|       ebook|
|          of|
|         the|
|    complete|
|       works|
|          of|
|     william|
|shakespeare,|
|          by|
|     william|
| shakespeare|
|            |
|        this|
|       ebook|
|          is|
|         for|
|         the|
+------------+
only showing top 20 rows



In [11]:

words = words.filter(col("word").rlike("^[a-z]"))


letters = words.select(
    substring(col("word"), 1, 1).alias("letter")
)

letters.show()

+------+
|letter|
+------+
|     t|
|     p|
|     g|
|     e|
|     o|
|     t|
|     c|
|     w|
|     o|
|     w|
|     s|
|     b|
|     w|
|     s|
|     t|
|     e|
|     i|
|     f|
|     t|
|     u|
+------+
only showing top 20 rows



In [12]:


result = letters.groupBy("letter").count().orderBy("letter")

result.show(26, truncate=False)

+------+------+
|letter|count |
+------+------+
|a     |84836 |
|b     |45455 |
|c     |34567 |
|d     |29713 |
|e     |18697 |
|f     |36814 |
|g     |20782 |
|h     |60563 |
|i     |62167 |
|j     |3339  |
|k     |9418  |
|l     |29569 |
|m     |55676 |
|n     |26759 |
|o     |43494 |
|p     |27759 |
|q     |2377  |
|r     |14265 |
|s     |65705 |
|t     |123602|
|u     |9170  |
|v     |5728  |
|w     |59597 |
|x     |14    |
|y     |25855 |
|z     |71    |
+------+------+



Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!